<a href="https://colab.research.google.com/github/aadityadamle/Keyword-Extraction-with-Bert/blob/main/Keyword_Extraction_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Description

This main objective of this project is to show the use of Keyword extraction using BERT. BERT is an unmatched tool showing an admiringly well performance for keyword/keyphrase extraction tasks. We will use BERT to extract good quality keyphrases from highly rank ads which can be used to generate more ads of optimum quality. The quality of the ad is indicated by the rank provided by Google to that particular ad.         

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools

In [4]:
!wget https://raw.githubusercontent.com/aadityadamle/Keyword-Extraction-with-Bert/main/flights_tickets_serp2020-03-15.csv
!wget https://raw.githubusercontent.com/aadityadamle/Keyword-Extraction-with-Bert/main/flights_tickets_serp2020-04-01.csv

--2020-11-18 19:36:31--  https://raw.githubusercontent.com/aadityadamle/Keyword-Extraction-with-Bert/main/flights_tickets_serp2020-03-15.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9487150 (9.0M) [text/plain]
Saving to: ‘flights_tickets_serp2020-03-15.csv’

flights_tickets_ser 100%[===================>]   9.05M  --.-KB/s    in 0.06s   

2020-11-18 19:36:32 (145 MB/s) - ‘flights_tickets_serp2020-03-15.csv’ saved [9487150/9487150]

--2020-11-18 19:36:32--  https://raw.githubusercontent.com/aadityadamle/Keyword-Extraction-with-Bert/main/flights_tickets_serp2020-04-01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151

#Dataset Description:
This dataset contains Google rankings for flights and tickets keywords. 
A single csv contains data of 100 destinations in 2 Keyword variations (flights to destination and tickets to destination) for 2 countries.
The file has 10 results for each combination ranked from 1 to 10 according to google rankings which makes a total of 4000 records per file. 

The same data set is produced every 15 days, in order to track the changes in ranks over time. Each file represents a point in time (either the 1st or the 15th of the month). We have taken 2 successive files, one of which will be used to obtain keywords and the other will be used to test the keywords.

The dataset used in this tutorial is available in public domain on kaggle:
 https://www.kaggle.com/eliasdabbas/search-engine-results-flights-tickets-keywords

In [5]:
#Examine the first csv
data1 = pd.read_csv("/content/flights_tickets_serp2020-03-15.csv")
data1.head() 

,searchTerms,rank,title,snippet,displayLink,link,queryTime,totalResults,cacheId,formattedUrl,htmlFormattedUrl,htmlSnippet,htmlTitle,kind,pagemap,cseName,count,startIndex,inputEncoding,outputEncoding,safe,cx,gl,searchTime,formattedSearchTime,formattedTotalResults
0,flights to hong kong,1,Flights to Hong Kong (HKG) | Cathay Pacific,Find & book great deals on Hong Kong (HKG) Fli...,www.cathaypacific.com,https://www.cathaypacific.com/destinations/en_...,2020-03-15 11:25:49.732215+00:00,222000000,UEdjkoYlsQsJ,https://www.cathaypacific.com/destinations/en_...,https://www.cathaypacific.com/destinations/en_...,Find &amp; book great deals on <b>Hong Kong</b...,<b>Flights to Hong Kong</b> (HKG) | Cathay Pac...,customsearch#result,{'cse_thumbnail': [{'src': 'https://encrypted-...,PySearch,10,1,utf8,utf8,off,012859022920491477448:pubdbfjmmec,us,0.336997,0.34,"222,000,000"
1,flights to hong kong,2,"$103 Flights to Hong Kong, China (HKG) - Tripa...",Book Cheap Flights to Hong Kong: Search and co...,www.tripadvisor.com,https://www.tripadvisor.com/Flights-g294217-Ho...,2020-03-15 11:25:49.732215+00:00,222000000,5jsjhmxgxvkJ,https://www.tripadvisor.com/Flights-g294217-Ho...,https://www.tripadvisor.com/<b>Flights</b>-g29...,Book Cheap <b>Flights to Hong Kong</b>: Search...,"$103 <b>Flights to Hong Kong</b>, China (HKG) ...",customsearch#result,{'cse_thumbnail': [{'src': 'https://encrypted-...,PySearch,10,1,utf8,utf8,off,012859022920491477448:pubdbfjmmec,us,0.336997,0.34,"222,000,000"
2,flights to hong kong,3,Cheap Flights to Hong Kong (HKG) from $437 - K...,"In general from United States, American Airlin...",www.kayak.com,https://www.kayak.com/flight-routes/United-Sta...,2020-03-15 11:25:49.732215+00:00,222000000,W42baDpas_gJ,https://www.kayak.com/flight-routes/United-Sta...,https://www.kayak.com/<b>flight</b>-routes/Uni...,"In general from United States, American Airlin...",Cheap <b>Flights to Hong Kong</b> (HKG) from $...,customsearch#result,{'cse_thumbnail': [{'src': 'https://encrypted-...,PySearch,10,1,utf8,utf8,off,012859022920491477448:pubdbfjmmec,us,0.336997,0.34,"222,000,000"
3,flights to hong kong,4,"Cheap Flights to Hong Kong SAR, Asia (HKG Airp...",Looking for cheap flights to Hong Kong SAR fro...,www.expedia.com,https://www.expedia.com/Cheap-Flights-To-Hong-...,2020-03-15 11:25:49.732215+00:00,222000000,iEmRbX0P8CAJ,https://www.expedia.com/Cheap-Flights-To-Hong-...,https://www.expedia.com/Cheap-<b>Flights-To-Ho...,Looking for cheap <b>flights to Hong Kong</b> ...,"Cheap <b>Flights to Hong Kong</b> SAR, Asia (H...",customsearch#result,{'cse_thumbnail': [{'src': 'https://encrypted-...,PySearch,10,1,utf8,utf8,off,012859022920491477448:pubdbfjmmec,us,0.336997,0.34,"222,000,000"
4,flights to hong kong,5,$397 Cheap Flights to Hong Kong Intl in 2020 |...,Find cheap flights to Hong Kong Intl (HKG) in ...,www.skyscanner.com,https://www.skyscanner.com/flights-to/hkg/chea...,2020-03-15 11:25:49.732215+00:00,222000000,YU4I4h8FLkwJ,https://www.skyscanner.com/flights-to/hkg/chea...,https://www.skyscanner.com/<b>flights-to/hk</b...,Find cheap <b>flights to Hong Kong</b> Intl (H...,$397 Cheap <b>Flights to Hong Kong</b> Intl in...,customsearch#result,{'metatags': [{'msapplication-tilecolor': '#07...,PySearch,10,1,utf8,utf8,off,012859022920491477448:pubdbfjmmec,us,0.336997,0.34,"222,000,000"


In [6]:
data1.columns

Index(['searchTerms', 'rank', 'title', 'snippet', 'displayLink', 'link',
       'queryTime', 'totalResults', 'cacheId', 'formattedUrl',
       'htmlFormattedUrl', 'htmlSnippet', 'htmlTitle', 'kind', 'pagemap',
       'cseName', 'count', 'startIndex', 'inputEncoding', 'outputEncoding',
       'safe', 'cx', 'gl', 'searchTime', 'formattedSearchTime',
       'formattedTotalResults'],
      dtype='object')

Now we will keep only those columns which are required for our analysis. Our objective of analysis is to generate keywords from ad - descriptions which can be used to make more highly ranked ad - descriptions in the future.  This only requires the ad - descriptions which are present in the snippet column. We also require the rank column which provides google ranking for that particular snippet. So we will remove all the other columns which are of no use.  

In [7]:
# Keeping only the required columns
dset1 = data1.iloc[:,:4]
dset1.head()

,searchTerms,rank,title,snippet
0,flights to hong kong,1,Flights to Hong Kong (HKG) | Cathay Pacific,Find & book great deals on Hong Kong (HKG) Fli...
1,flights to hong kong,2,"$103 Flights to Hong Kong, China (HKG) - Tripa...",Book Cheap Flights to Hong Kong: Search and co...
2,flights to hong kong,3,Cheap Flights to Hong Kong (HKG) from $437 - K...,"In general from United States, American Airlin..."
3,flights to hong kong,4,"Cheap Flights to Hong Kong SAR, Asia (HKG Airp...",Looking for cheap flights to Hong Kong SAR fro...
4,flights to hong kong,5,$397 Cheap Flights to Hong Kong Intl in 2020 |...,Find cheap flights to Hong Kong Intl (HKG) in ...


In [8]:
# Select the records with rank less than 3. Select the records with rank less 
# 3 to select the top 3 results for each search term.

dset1 = dset1.where(dset1["rank"] < 4)
print(dset1.shape)
dset1.head()

(4000, 4)


,searchTerms,rank,title,snippet
0,flights to hong kong,1.0,Flights to Hong Kong (HKG) | Cathay Pacific,Find & book great deals on Hong Kong (HKG) Fli...
1,flights to hong kong,2.0,"$103 Flights to Hong Kong, China (HKG) - Tripa...",Book Cheap Flights to Hong Kong: Search and co...
2,flights to hong kong,3.0,Cheap Flights to Hong Kong (HKG) from $437 - K...,"In general from United States, American Airlin..."
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [9]:
# Remove records with NAN values
dset1 = dset1.dropna()
print(dset1.shape)
dset1.head()

(1200, 4)


,searchTerms,rank,title,snippet
0,flights to hong kong,1.0,Flights to Hong Kong (HKG) | Cathay Pacific,Find & book great deals on Hong Kong (HKG) Fli...
1,flights to hong kong,2.0,"$103 Flights to Hong Kong, China (HKG) - Tripa...",Book Cheap Flights to Hong Kong: Search and co...
2,flights to hong kong,3.0,Cheap Flights to Hong Kong (HKG) from $437 - K...,"In general from United States, American Airlin..."
10,flights to bangkok,1.0,"$24 Flights to Bangkok, Thailand (BKK) - Tripa...",Book Cheap Flights to Bangkok: Search and comp...
11,flights to bangkok,2.0,"$267 - Cheap Flights to Bangkok, Thailand in 2...",Looking for cheap one way flights ($267) to Ba...


In [10]:
# Create a series of all the ad - descriptions (snippets)

snippets = dset1.iloc[:,3]
print(snippets[0])

Find & book great deals on Hong Kong (HKG) Flights. Fly to Hong Kong with 
Cathay Pacific and experience award-winning service.


In [11]:
snippets.count()

1200

In [12]:
# Remove stopwords and extract candidates (words or phrases). 
# Creation of candidates from existing snippets

n_gram_range = (3, 3)
stop_words = "english"

# Extract candidate words/phrases
list_of_candidates = []
for snippet in snippets:
  count = CountVectorizer(ngram_range=n_gram_range, 
                          stop_words=stop_words).fit([snippet])
  candidates = count.get_feature_names()
  #print(candidates)
  #create list of candidates where each candidates is a list of keywords
  list_of_candidates.append(candidates)
print(len(list_of_candidates))   #Expected 1200

1200


# Snippet and Candidates
Snippet in this project means an ad - description. Ad - descriptions are present in the column named snippet in the dataset. After accessig the dataset we have created a series of ad - descriptions and named it as snippets.

Then we have used the CountVectorizer from sklearn.feature_extraction.text lib to extract 5 phrases of 3 words each from each snippet to form the Candidates. We will perform futher processing on these candidates to obtain our keywords. So basically the candidates are premature versions of the final keywords.





In [13]:
#print first candidates list from list_of_candidates
print(list_of_candidates[0])

['award winning service', 'book great deals', 'cathay pacific experience', 'deals hong kong', 'experience award winning', 'flights fly hong', 'fly hong kong', 'great deals hong', 'hkg flights fly', 'hong kong cathay', 'hong kong hkg', 'kong cathay pacific', 'kong hkg flights', 'pacific experience award']


Now we convert both the snippets as well as the candidate keywords/keyphrases to numerical data. We use BERT for this purpose.
For generating the BERT embeddings, we will use the sentence-transformers package. It allows the user to quickly create high-quality embeddings that work appropriately for sentence - level embeddings.
We install the package with pip install sentence-transformers. If you run into issues installing this package, then it might be helpful to install Pytorch first.

In [14]:
# install sentence transformers
!pip install sentence-transformers

     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 1.3MB 21.1MB/s 
     |████████████████████████████████| 890kB 56.2MB/s 
     |████████████████████████████████| 2.9MB 55.4MB/s 
     |████████████████████████████████| 1.1MB 55.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=58b95d8a09280614985fd3b327ece817bd443b83ffb501151ea9c2e18ff9e383
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=67699f1de35aedf79f92bf8cc21c6778df37b497ffce3ab2010ff3405d0d91e8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [15]:
# Use sentence transformers to create embeddings of both snippets and candidates 

#create lists for embedded text and candidates
embedded_snippets = []
embedded_list_of_candidates = []

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Taking snippet from list of snippets and creating new list of embedded 
# snippets
for snippet in snippets:
  snippet_embedding = model.encode([snippet])
  embedded_snippets.append(snippet_embedding)

# Taking candidates from list of candidates and creating new list of embedded 
# candidates

for candidates in list_of_candidates:
  candidate_embeddings = model.encode(candidates)
  embedded_list_of_candidates.append(candidate_embeddings)


print(len(embedded_snippets))              #Expected 1200
print(len(embedded_list_of_candidates))    #Expected 1200


100%|██████████| 245M/245M [00:11<00:00, 21.6MB/s]


1200
1200


# Cosine_similarity

Cosine similarity measures the cosine of the angle between two vectors projected in a multi-dimensional space. It is a measure of how similar the documents are which is independent of their size.  The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

Generally the similarity of documents is based on counting the maximum number of common words between the documents. But this approach tends to inaccuracy as the size of the document increases and the number of common words also increase even if the documents talk about different topics.

The cosine similarity is a better alternative for ‘count-the-common-words’ or Euclidean distance approach.




In [16]:
# Checking similary using cosine_similarity function and then selecting top 5 
# most similar candidates for the corresponding snippet. 
# These candidates are most similar to the snippet and thus best represent 
# the snippet. Finally we get 1200 set of candidates called as keywords  

top_n = 5
list_of_keywords = []
list_of_distances = []

for (embedded_snippet,
     embedded_candidates,
     candidates) in zip(embedded_snippets,
                        embedded_list_of_candidates,
                        list_of_candidates):
       
  distances = cosine_similarity(embedded_snippet,
                                embedded_candidates)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
  #print(keywords)
  list_of_keywords.append(keywords)

print(len(list_of_keywords))  

1200


In [17]:
# Now we check the similarity of candidates with snippet and with 4 other 
# candidates of the same snippet. The goal is to obtain candidates which 
# are most similar to snippet but which are least similar to one another. 
# This provides diversity to our list of keywords 

def max_sum_sim(embedded_snippet,
                embedded_candidates,
                candidates,
                top_n,
                nr_candidates):
  # Calculate distances and extract keywords
  distances = cosine_similarity(embedded_snippet,
                                embedded_candidates)
  distances_candidates = cosine_similarity(embedded_candidates,
                                           embedded_candidates)
  
  # Get top_n words as candidates based on cosine similarity
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]
  
  # Calculate the combination of words that are the least similar to each other
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] 
               for i in combination 
               for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim
        
  return[words_vals[idx] for idx in candidate]

In [18]:
# Now use the above function to obtain keywords with maximum candidate-snippet 
# similarity and minimum candidate-candidate similarity. At last make a huge 
# list of keyword which can be used to create ad - descriptions in future 
# which will have high rank.
  


list_of_keywords = []
keywords = []
for (embedded_snippet,
     embedded_candidates,
     candidates) in zip(embedded_snippets,
                        embedded_list_of_candidates,
                        list_of_candidates):
       
  keywords = max_sum_sim(embedded_snippet,
                         embedded_candidates,
                         candidates,
                         5,
                         10)
  
  list_of_keywords.append(keywords)


print(list_of_keywords)
print(len(list_of_keywords))     #Expected 1200

# Now make a huge list of keywords by appending all lists
all_keywords = []

for keywords in list_of_keywords:
  for i in keywords:
    all_keywords.append(i)

print(len(all_keywords))  #Expected 1200 * 5 = 6000


[['hkg flights fly', 'cathay pacific experience', 'hong kong cathay', 'book great deals', 'great deals hong'], ['kong choose best', 'search compare airfares', 'cheap flights hong', 'book cheap flights', 'tripadvisor best flights'], ['cathay pacific malaysia', 'kong popular route', 'popular route new', 'states american airlines', 'american airlines cathay'], ['flights bangkok search', 'airfares tripadvisor best', 'book cheap flights', 'trip bangkok choose', 'choose best airline'], ['jetblue frontier compare', 'airlines american airlines', 'cheap way flights', 'american airlines delta', 'flights 267 bangkok'], ['bangkok compare prices', 'major airlines delta', 'looking cheap flights', 'travel agencies cheap', 'hundreds major airlines'], ['way flights 274', 'available past days', '174 way flights', 'london prices available', 'cheapest flight london'], ['trip london choose', 'flights london search', 'book cheap flights', 'airfares tripadvisor best', 'tripadvisor best flights'], ['canada fl


# Thus we have obtained 6000 key phrases which can be used to make ads which will be highly ranked by search engines. 

Now we will load the second csv file and check whether the keywords in all_keywords list are present in ad - descriptions of the second file. Then we will also check the rank of those ad - descriptions. 

In [19]:
data2 = pd.read_csv("/content/flights_tickets_serp2020-04-01.csv")
dset2 = data2.iloc[:,1:4]
dset2.drop("title", axis = "columns", inplace= True)
print(dset2.head())
dset2.shape

   rank                                            snippet
0     1  Book Cheap Flights to Hong Kong: Search and co...
1     2  Find & book great deals on Hong Kong (HKG) Fli...
2     3  Looking for cheap flights to Hong Kong SAR fro...
3     4  Find flights to Hong Kong on China Eastern, As...
4     5  Find cheap flights to Hong Kong Intl (HKG) in ...


(4000, 2)

In [20]:
snippets_2 = dset2.values.tolist()
print(snippets_2)

[[1, 'Book Cheap Flights to Hong Kong: Search and compare airfares on Tripadvisor \nto find the best flights for your trip to Hong Kong. Choose the best airline for you\xa0...'], [2, 'Find & book great deals on Hong Kong (HKG) Flights. Fly to Hong Kong with \nCathay Pacific and experience award-winning service.'], [3, 'Looking for cheap flights to Hong Kong SAR from airlines such as American \nAirlines, Delta, United, JetBlue, Frontier, and more? Compare prices on Expedia \nto\xa0...'], [4, 'Find flights to Hong Kong on China Eastern, Asiana Airlines, United Airlines and \nmore. Fly round-trip from Chicago from $487, from Seattle from $502, from\xa0...'], [5, "Find cheap flights to Hong Kong Intl (HKG) in 2019 using Skyscanner's flight \ncomparison tool. We compare all major airlines and travel agents for flights to \nHong\xa0..."], [6, 'Looking for cheap flights to Hong Kong from your destination? Search for airfare \nand flight ticket deals at cheapflights.com and book your next flig

In [21]:
# Check whether a particular keyword from a huge list of 6000 keywords is 
# present in an ad from list of 4000 ads. If yes then append the details 
# in appropriate lists. There can be 24 x 10^6 combination possible.

list_of_ads = []
list_of_ranks = []
list_of_keywords = []

for (rank, snippet) in snippets_2:
  for keyword in all_keywords:
    if keyword in snippet:
      list_of_ads.append(snippet)
      list_of_ranks.append(rank)
      list_of_keywords.append(keyword)

In [22]:
#Generate a dataframe to display the lists generated in previous step 

ad_results = pd.DataFrame(list(zip(list_of_keywords,
                                   list_of_ads,
                                   list_of_ranks)), 
               columns =['Keywords', 'Ads', 'Ranks']) 
ad_results 


,Keywords,Ads,Ranks
0,book great deals,Find & book great deals on Hong Kong (HKG) Fli...,2
1,flight ticket deals,Looking for cheap flights to Hong Kong from yo...,6
2,flight ticket deals,Looking for cheap flights to Hong Kong from yo...,6
3,flight ticket deals,Looking for cheap flights to Hong Kong from yo...,6
4,flight ticket deals,Looking for cheap flights to Hong Kong from yo...,6
...,...,...,...
18846,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5
18847,uk offers cheap,Expedia.co.uk offers cheap flights to Nice. Fi...,5
18848,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5
18849,uk offers cheap,Expedia.co.uk offers cheap flights to Nice. Fi...,5


In [23]:
# check the records for particular rank

rank_3 = ad_results.where(ad_results["Ranks"] == 3).dropna()
rank_3

,Keywords,Ads,Ranks
82,flight ticket deals,"Looking for cheap flights to Paris, France fro...",3.0
83,flight ticket deals,"Looking for cheap flights to Paris, France fro...",3.0
84,flight ticket deals,"Looking for cheap flights to Paris, France fro...",3.0
85,flight ticket deals,"Looking for cheap flights to Paris, France fro...",3.0
86,flight ticket deals,"Looking for cheap flights to Paris, France fro...",3.0
...,...,...,...
18533,cheapest return tickets,Which airlines provide the cheapest flights fr...,3.0
18534,cheapest return tickets,Which airlines provide the cheapest flights fr...,3.0
18535,cheapest return tickets,Which airlines provide the cheapest flights fr...,3.0
18536,cheapest return tickets,Which airlines provide the cheapest flights fr...,3.0


In [24]:
# check the records for particular keyword

phrase_1 = ad_results.where(ad_results["Keywords"] == "offers cheap flights").dropna()
phrase_1

,Keywords,Ads,Ranks
148,offers cheap flights,Expedia.co.uk offers cheap flights to Mecca. F...,7.0
150,offers cheap flights,Expedia.co.uk offers cheap flights to Mecca. F...,7.0
151,offers cheap flights,Expedia.co.uk offers cheap flights to Mecca. F...,7.0
153,offers cheap flights,Expedia.co.uk offers cheap flights to Mecca. F...,7.0
155,offers cheap flights,Expedia.co.uk offers cheap flights to Mecca. F...,7.0
...,...,...,...
18843,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5.0
18844,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5.0
18846,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5.0
18848,offers cheap flights,Expedia.co.uk offers cheap flights to Nice. Fi...,5.0


In [25]:
# Obtain the lists of unique values in columns
print(ad_results.Keywords.unique())
print(len(ad_results.Keywords.unique()))

['book great deals' 'flight ticket deals' 'preferred departure airport'
 'best flight deals' 'major domestic airlines' 'uk offers cheap'
 'offers cheap flights' 'travel guide pages' '545 round trip'
 'best price guarantee' 'lowest airfare flights' 'book flight tickets'
 'offer direct flights' 'airlines offer direct' 'flight comparison tool'
 'availability change rapidly' 'seat availability change'
 'low cost flights' 'book cheap flights' 'book cheap flight'
 'daily scheduled flight' '11th busiest airport'
 'official international airport' 'offering direct flights'
 'cruise tour packages' 'compare flight prices' 'search multiple sites'
 'cost airline prices' 'cheapest low cost' 'browse flight prices'
 'cheapest flight ticket' 'international airlines including'
 'award winning hospitality' 'average flight time' 'book flight deals'
 'closest major airport' 'available year round' 'fifth largest city'
 'finding cheap flights' 'trip discounted tickets' 'regular fare tickets'
 'cheap flight t

# So we have found 73 unique key phrases that can be used for ads in the future. By checking the records for particular ranks as shown earlier we can obtain  the key phrases for highly ranked ads. 